In [1]:
import pandas as pd
import glob
import os
import warnings


In [2]:
# Use glob to find all Excel files starting with 'Content_'
file_paths = glob.glob('Content_*.xlsx')

In [3]:
sheet_name = "ENGAGEMENT"

# Clean and merge ENGAGEMENT sheets
engagement_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(f, sheet_name=sheet_name)

    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    engagement_dfs.append(df)

print(engagement_dfs[0].head(5))



c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


         Date  Impressions  Engagements
0   11/9/2024            8            0
1  11/10/2024           15            0
2  11/11/2024           16            0
3  11/12/2024            5            0
4  11/13/2024           13            1


In [4]:
merged_engagement_df = pd.concat(engagement_dfs)
date_col = merged_engagement_df.columns[0]
merged_engagement_df[date_col] = pd.to_datetime(merged_engagement_df[date_col], errors="coerce")
merged_cleaned_engagement_df = merged_engagement_df.drop_duplicates(subset=date_col)
merged_cleaned_engagement_df = merged_cleaned_engagement_df.sort_values(by=date_col)
print(merged_engagement_df.head(5))


# merged_engagement_df.to_excel('engagement.xlsx', index=False)



        Date  Impressions  Engagements
0 2024-11-09            8            0
1 2024-11-10           15            0
2 2024-11-11           16            0
3 2024-11-12            5            0
4 2024-11-13           13            1


In [5]:
len(merged_engagement_df)

464

In [6]:
sheet_name = "FOLLOWERS"
followers_dfs = []

for f in file_paths:
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
        skiprows=2
    )

    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    followers_dfs.append(df)

print(followers_dfs[0].head(5))

c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


        Date  New followers
0 2024-11-09              0
1 2024-11-10              0
2 2024-11-11              2
3 2024-11-12              2
4 2024-11-13              2


In [7]:
followers_merged = pd.concat(followers_dfs, ignore_index=True)
followers_merged = followers_merged.drop_duplicates(subset=date_col)
followers_merged = followers_merged.sort_values(by=date_col)
print(followers_merged.head(5))
# followers_merged.to_excel('followers.xlsx', index=False)

        Date  New followers
0 2024-11-09              0
1 2024-11-10              0
2 2024-11-11              2
3 2024-11-12              2
4 2024-11-13              2


In [8]:
len(followers_merged)

400

In [9]:
sheet_name = "DEMOGRAPHICS"

demographics_dfs = []

for i, f in enumerate(file_paths):
    df = pd.read_excel(
        f,
        sheet_name=sheet_name,
    )
    # keep header only from first file
    if i > 0:
        df = df.iloc[1:]

    demographics_dfs.append(df)

print(demographics_dfs[0].head(5))



    

  Top Demographics              Value Percentage
0       Job titles   Technical Writer   0.259905
1       Job titles            Founder   0.024344
2       Job titles  Software Engineer    0.02315
3       Job titles    Product Manager   0.014081
4       Job titles         Co-Founder   0.013126


c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
# Merge all demographics DataFrames first
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

# merged_demographics_df.to_excel('demographics.xlsx', index=False)


  Top Demographics              Value  Percentage
0       Job titles   Technical Writer    0.002599
1       Job titles            Founder    0.000243
2       Job titles  Software Engineer    0.000232
3       Job titles    Product Manager    0.000141
4       Job titles         Co-Founder    0.000131
Top Demographics     object
Value                object
Percentage          float64
dtype: object


In [24]:
merged_demographics_df = pd.concat(demographics_dfs, ignore_index=True)

pct_col = "Percentage"
s = merged_demographics_df[pct_col].astype(str).str.strip()

# extract numeric part from:
# "< 1%", ">95%", "23%", "23.5", "< 0.5 %", etc.
s = s.str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False)

# convert to decimal percentage
merged_demographics_df[pct_col] = pd.to_numeric(s, errors="raise") / 100

merged_demographics_df = merged_demographics_df.drop_duplicates(subset="Value")

print(merged_demographics_df.head())
print(merged_demographics_df.dtypes)

merged_demographics_df.to_excel('demographics.xlsx', index=False)


  Top Demographics              Value  Percentage
0       Job titles   Technical Writer    0.002599
1       Job titles            Founder    0.000243
2       Job titles  Software Engineer    0.000232
3       Job titles    Product Manager    0.000141
4       Job titles         Co-Founder    0.000131
Top Demographics     object
Value                object
Percentage          float64
dtype: object


In [75]:
len(merged_demographics_df)

30

In [12]:


sheet_name = "TOP POSTS"

top_engagement_dfs = []

for f in file_paths:
    # read without treating any row as header
    df = pd.read_excel(f, sheet_name=sheet_name, header=None, skiprows=2)
    top_posts_engagement = df.iloc[:,0:3].copy()
    top_posts_engagement.columns = [
        "Post URL", "Post publish date", "Engagement"
    ]
    
  
    top_posts_engagement = top_posts_engagement.dropna(how="all")
    top_posts_engagement = top_posts_engagement.fillna(0)
    top_engagement_dfs.append(top_posts_engagement)
    combined_top_engagement = pd.concat(top_engagement_dfs, ignore_index=True)

print(combined_top_engagement.head(5))

   

                                            Post URL  Post publish date  \
0                                           Post URL  Post publish date   
1  https://www.linkedin.com/feed/update/urn:li:ac...          6/16/2025   
2  https://www.linkedin.com/feed/update/urn:li:ac...           8/1/2025   
3  https://www.linkedin.com/feed/update/urn:li:ac...          6/12/2025   
4  https://www.linkedin.com/feed/update/urn:li:ac...          7/19/2025   

    Engagement  
0  Engagements  
1          464  
2          439  
3          414  
4          371  


c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
len(combined_top_engagement)

135

In [14]:
combined_top_engagement["Engagement"] = (pd.to_numeric(combined_top_engagement["Engagement"], errors="coerce").fillna(0).astype(int))
combined_top_engagement["Post publish date"] = (pd.to_datetime(combined_top_engagement["Post publish date"], errors="coerce"))
combined_top_engagement = combined_top_engagement.sort_values(by="Post publish date")

print(combined_top_engagement.info(5))

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 48 to 123
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Post URL           135 non-null    object        
 1   Post publish date  131 non-null    datetime64[ns]
 2   Engagement         135 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.2+ KB
None


C:\Users\Steve Jose\AppData\Local\Temp\ipykernel_20988\2202816473.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_top_engagement["Post publish date"] = (pd.to_datetime(combined_top_engagement["Post publish date"], errors="coerce"))


In [15]:
print(top_posts_engagement.shape)
print(top_posts_engagement.info())


(12, 3)
<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Post URL           12 non-null     object
 1   Post publish date  12 non-null     object
 2   Engagement         12 non-null     object
dtypes: object(3)
memory usage: 384.0+ bytes
None


In [16]:
sheet_name = "TOP POSTS"

top_impressions_dfs = []

for f in file_paths: 
    df = pd.read_excel(f, sheet_name=sheet_name, header=None, skiprows=3)

    top_posts_impressions = df.iloc[:,4:7].copy()
    top_posts_impressions.columns = ["Post URL", "Post publish date", "Impressions"] 

    top_posts_impressions = top_posts_impressions.dropna(how="all")

    top_posts_impressions = top_posts_impressions.fillna(0)

    top_impressions_dfs.append(top_posts_impressions)

    combined_top_impressions = pd.concat(top_impressions_dfs, ignore_index=True)

    print(combined_top_impressions.head(5))

                                            Post URL Post publish date  \
0  https://www.linkedin.com/feed/update/urn:li:ac...         7/19/2025   
1  https://www.linkedin.com/feed/update/urn:li:ac...         6/16/2025   
2  https://www.linkedin.com/feed/update/urn:li:ac...         6/27/2025   
3  https://www.linkedin.com/feed/update/urn:li:ac...         6/12/2025   
4  https://www.linkedin.com/feed/update/urn:li:ac...         5/15/2025   

   Impressions  
0        18928  
1        16531  
2        12046  
3        11800  
4        10501  
                                            Post URL Post publish date  \
0  https://www.linkedin.com/feed/update/urn:li:ac...         7/19/2025   
1  https://www.linkedin.com/feed/update/urn:li:ac...         6/16/2025   
2  https://www.linkedin.com/feed/update/urn:li:ac...         6/27/2025   
3  https://www.linkedin.com/feed/update/urn:li:ac...         6/12/2025   
4  https://www.linkedin.com/feed/update/urn:li:ac...         5/15/2025   

   Impre

c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Steve Jose\Documents\Link\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
len(combined_top_impressions)

198

In [18]:
combined_top_impressions["Impressions"] = (pd.to_numeric(combined_top_impressions["Impressions"], errors="coerce").fillna(0).astype(int))
combined_top_impressions["Post publish date"] = (pd.to_datetime(combined_top_impressions["Post publish date"], errors="coerce"))

combined_top_impressions = combined_top_impressions.sort_values(by="Post publish date")

dupe_rows_eng = combined_top_impressions["Post publish date"].duplicated(keep=False)
has_dupe_eng = dupe_rows_eng.any()
print("Engagement DF has duplicate dates:", has_dupe_eng)
# print(combined_top_impressions.info(5))

duplicates = combined_top_impressions[dupe_rows_eng]
print(duplicates.sort_values("Post publish date"))

# duplicates.to_excel("duplicates.xlsx", index=False)

Engagement DF has duplicate dates: True
                                              Post URL Post publish date  \
36   https://www.linkedin.com/feed/update/urn:li:ac...        2025-03-26   
196  https://www.linkedin.com/feed/update/urn:li:ac...        2025-03-26   
192  https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-11   
141  https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-11   
94   https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-11   
..                                                 ...               ...   
59   https://www.linkedin.com/feed/update/urn:li:ac...        2025-12-09   
150  https://www.linkedin.com/feed/update/urn:li:ac...        2025-12-09   
103  https://www.linkedin.com/feed/update/urn:li:ac...        2025-12-12   
75   https://www.linkedin.com/feed/update/urn:li:ac...        2025-12-12   
152  https://www.linkedin.com/feed/update/urn:li:ac...        2025-12-12   

     Impressions  
36          2410  
196      

In [19]:
df_dup = duplicates.copy()

df_dup["Post publish date"] = pd.to_datetime(df_dup["Post publish date"], errors="coerece")

df_dup["Impressions"] = pd.to_numeric(df_dup["Impressions"], errors="coerce").fillna(0).astype(int)

df_dup = df_dup.sort_values(["Post URL", "Impressions"], ascending=[True, False])

canonical_impressions = (
    df_dup
    .groupby("Post URL", as_index=False)
    .first()
)

print(canonical_impressions.head(5))
canonical_impressions.to_excel('impressions.xlsx', index=False)

                                            Post URL Post publish date  \
0  https://www.linkedin.com/feed/update/urn:li:ac...        2025-03-26   
1  https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-11   
2  https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-15   
3  https://www.linkedin.com/feed/update/urn:li:ac...        2025-05-22   
4  https://www.linkedin.com/feed/update/urn:li:ac...        2025-06-01   

   Impressions  
0         2410  
1         9968  
2        10501  
3         3098  
4         2707  


In [20]:
len(canonical_impressions)

53

In [31]:
import pandas as pd

df = followers_merged.copy()

# ensure datetime and proper numeric type
df["Date"] = pd.to_datetime(df["Date"])
df["New followers"] = pd.to_numeric(df["New followers"], errors="coerce")

# time features
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)      # e.g. "2025-09"
df["Quarter"] = df["Date"].dt.to_period("Q").astype(str)    # e.g. "2025Q3"

# cumulative followers (assuming data is in chronological order)
df = df.sort_values("Date")
df["Total followers"] = df["New followers"].cumsum()

# daily pct change in total followers
df["Total followers pct_change"] = df["Total followers"].pct_change()

# monthly aggregates
monthly = (
    df.groupby("Month")
      .agg(
          new_followers=("New followers", "sum"),
          total_followers_end=("Total followers", "last"),
      )
      .reset_index()
)
monthly["total_followers_pct_change"] = monthly["total_followers_end"].pct_change()


In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    suppress_callback_exceptions=True,
)

years = sorted(df["Year"].unique())

app.layout = dbc.Container(
    [
        # HEADER
        dbc.Row(
            dbc.Col(
                html.Div(
                    [
                        html.H2("Followers Growth Dashboard", className="mb-0"),
                        html.P("LinkedIn audience growth over time", className="mb-0"),
                    ],
                    className="app-header",
                ),
                width=12,
            ),
            className="mb-4",
        ),

        # FILTERS
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Year", className="filter-label"),
                        dcc.Dropdown(
                            id="year-dropdown",
                            options=[{"label": y, "value": y} for y in years],
                            value=years[-1],
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
                dbc.Col(
                    [
                        html.Label("Granularity", className="filter-label"),
                        dcc.Dropdown(
                            id="freq-dropdown",
                            options=[
                                {"label": "Daily", "value": "D"},
                                {"label": "Monthly", "value": "M"},
                                {"label": "Quarterly", "value": "Q"},
                            ],
                            value="M",
                            clearable=False,
                            className="filter-dropdown",
                        ),
                    ],
                    md=3,
                ),
            ],
            className="mb-3",
        ),

        # KPI CARDS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Total followers", className="card-title-text"),
                                html.H3(id="total-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-primary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("Followers this period", className="card-title-text"),
                                html.H3(id="period-followers-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-secondary",
                    ),
                    md=4,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P("% change vs previous", className="card-title-text"),
                                html.H3(id="pct-change-card", className="card-kpi"),
                            ]
                        ),
                        className="kpi-card kpi-card-accent",
                    ),
                    md=4,
                ),
            ],
            className="mb-4",
        ),

        # CHARTS
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Cumulative followers", className="chart-title"),
                                dcc.Graph(id="cum-line", config={"displayModeBar": False}),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
                dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("New followers by period", className="chart-title"),
                                dcc.Graph(id="bar-new-followers", config={"displayModeBar": False}),
                            ]
                        ),
                        className="chart-card",
                    ),
                    md=6,
                ),
            ]
        ),
    ],
    fluid=True,
    className="app-container",
)


@app.callback(
    [
        Output("total-followers-card", "children"),
        Output("period-followers-card", "children"),
        Output("pct-change-card", "children"),
        Output("cum-line", "figure"),
        Output("bar-new-followers", "figure"),
    ],
    [Input("year-dropdown", "value"), Input("freq-dropdown", "value")],
)
def update_dashboard(year, freq):
    dff = df[df["Year"] == year].copy()
    dff = dff.sort_values("Date")

    total_followers = dff["Total followers"].iloc[-1]

    if freq == "D":
        agg = dff.copy()
        x_col = "Date"
        agg_new = agg["New followers"].sum()
        pct = dff["Total followers"].iloc[-1] / dff["Total followers"].iloc[0] - 1
    elif freq == "M":
        agg = (
            dff.groupby("Month")
               .agg(new_followers=("New followers", "sum"),
                    total_followers_end=("Total followers", "last"))
               .reset_index()
        )
        x_col = "Month"
        agg_new = agg["new_followers"].iloc[-1]
        pct = agg["total_followers_end"].pct_change().iloc[-1]
    else:  # "Q"
        agg = (
            dff.groupby("Quarter")
               .agg(new_followers=("New followers", "sum"),
                    total_followers_end=("Total followers", "last"))
               .reset_index()
        )
        x_col = "Quarter"
        agg_new = agg["new_followers"].iloc[-1]
        pct = agg["total_followers_end"].pct_change().iloc[-1]

    # line chart
    line_fig = px.line(
        dff,
        x="Date",
        y="Total followers",
        markers=True,
    )
    line_fig.update_layout(
        template="plotly_dark",
        margin=dict(l=10, r=10, t=10, b=10),
        xaxis_title=None,
        yaxis_title=None,
    )

    # bar chart
    bar_fig = px.bar(
        agg,
        x=x_col,
        y="new_followers",
    )
    bar_fig.update_layout(
        template="plotly_dark",
        margin=dict(l=10, r=10, t=10, b=10),
        xaxis_title=None,
        yaxis_title=None,
    )
    bar_fig.update_xaxes(tickangle=-35)

    return (
        f"{int(total_followers):,}",
        f"{int(agg_new):,}",
        f"{pct*100:0.1f}%",
        line_fig,
        bar_fig,
    )


if __name__ == "__main__":
    app.run(debug=True, port=8051)
